Install Dependencies


-	Најпрво се импортираат потребните библиотеки кои ќе ги користиме во понатамошното кодирање, а тие се: numpy и pandas.
Numpy -> едноставна, а моќна податочна структура, а тоа е н-дименсионална низа.
-Pandas -> библиотека за манипулација и анализа на податоци.
-	Потоа ги инсталираме трансформерите


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install transformers

Preprocessing


Во променлива text  ги сместуваме податоците од фајлот train_text.txt. Потоа во променлива tweet_train го прикажуваме целиот текст линија по линија и го печатиме.
Во label_trains ги сместуваме лабелите и потоа печатиме.
Во tweet_test ги сместуваме податоците од test_text.txt и истите ги печатиме. Истата постапка ја правиме и со следните променливи: label_test, tweet_val, labels_val.
-	Дефинираме функција preprocess_label(y) – правиме чистење на податоците
-	Во променливата data ја сместуваме табелата која што ја креираме преку pd.DataFrame. Табелата е составена од две колони во кои се прикажани tweet_train податоците и tweet_label_train податоците. А со слесдниот ред data.head() ги прикажуваме првите пет редови од табелата.


In [ ]:
text = open("/content/drive/MyDrive/train_text.txt", 'r')
tweet_train = text.readlines()

In [ ]:
tweet_train

In [ ]:
text = open('/content/drive/MyDrive/train_labels.txt', 'r')
labels_train = text.readlines()

In [ ]:
labels_train

In [ ]:
text = open('/content/drive/MyDrive/test_text.txt', 'r')
tweet_test = text.readlines()

In [ ]:
tweet_test

In [ ]:
text = open('/content/drive/MyDrive/test_labels.txt', 'r')
labels_test = text.readlines()

In [ ]:
labels_test

In [ ]:
text = open('/content/drive/MyDrive/val_text.txt', 'r')
tweet_val = text.readlines()

In [ ]:
tweet_val

In [ ]:
text = open('/content/drive/MyDrive/val_labels.txt', 'r')
labels_val = text.readlines()

In [ ]:
labels_val

In [ ]:
def preprocess_label(y):
  new_text = []
  for t in y:
    t = t.replace("\n", "")
    t = int(t)
    new_text.append(t)
  return (new_text)
tweet_labels_train = preprocess_label(labels_train)
tweet_labels_test = preprocess_label(labels_test)
tweet_labels_val = preprocess_label(labels_val)

In [ ]:
data = pd.DataFrame({'text': tweet_train, 'labels':tweet_labels_train})
data.head()

,text,labels
0,Sunday afternoon walking through Venice in the...,12
1,Time for some BBQ and whiskey libations. Chomp...,19
2,Love love love all these people ️ ️ ️ #friends...,0
3,"️ ️ ️ ️ @ Toys""R""Us \n",0
4,Man these are the funniest kids ever!! That fa...,2


# Loading Toknizer and Encoding our Data

- Во овој дел за токенизирање користиме готов BertTokenizer од transformers библиотеката за кој треба податоците да бидат добро средени за да може bert добро да ги трансформира. Исто така користиме и TensorDataset од torch.utils.data библиотеката. Ова го користиме со цел да ги претвориме нашите datasets во tensors.
- Моделот кој што го земаме е bert-based-uncased. Тој е првиот базичен модел кој е базиран на англиски јазик. Со do_lower_case=True кажуваме дека сакаме токените да бидат претворени во мали букви. Потоа од tokenizer со batch_encode_plus ги енкодираме зборовите според batch size, односно дел по дел. Special tokens ги немаме во текстовите затоа ги додаваме со add_special_tokens. Тие се за почеток на реченицата, за маскирање на зборовите и сепаратор за реченици. Return_attention_mask го поставуваме на Tru и тоа означува како се предвидуваат маскираните зборови. 
- Делот pad_to_max_length го поставуваме на True, и ни означува колку максимално да биде секој embedding, осносно векторската репрезентација. - На крајот враќаме tensor. Истото го правиме и за валидацискиот dataset. Потоа оваа енкодирана data ја делиме на 3 дела и тоа: input_ids, attention_mask и labels односно таргет колоната.


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [ ]:
import torch
encoded_data_train = tokenizer.batch_encode_plus(
    tweet_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    tweet_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(tweet_labels_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(tweet_labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


- Во овој дел ги правиме datasets за train и validation. Имаме TensorDataSet бидејќи примаме tensor на инпут.

In [ ]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [ ]:
dataset_val.tensors

(tensor([[  101,  1037,  2210,  ...,     0,     0,     0],
         [  101,  1043, 10278,  ...,     0,     0,     0],
         [  101,  7072,  8232,  ...,     0,     0,     0],
         ...,
         [  101,  2613,  2814,  ...,     0,     0,     0],
         [  101,  2009,  3084,  ...,     0,     0,     0],
         [  101,  2009,  1005,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([ 0,  7, 11,  ...,  6,  3,  2]))

# Setting up BERT Pretrained Model

- Од transformers го импортираме BertForSequenceClassification. Го користиме овој модел затоа што сакаме да направи класификација на секоја секвенца.
- Во овој дел правиме label dictionary односно ги земаме сите возможни labels.
- Тука земаме повторно претрениран модел на Bert. Го земаме основниот модел кој е на англиски јазик. Со num_labels го земаме бројот на labels.


In [ ]:
 from transformers import BertForSequenceClassification

In [ ]:
label_dict = {}
possible_labels = data['labels'].unique() 

for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

> **Creating Data Loaders**

- Тука дефинираме batch size и од библиотеката torch.utils.data ги вклучуваме DataLoader и RandomSampler. DataLoader ги чита податоците така што ги трансформира за потребите на torch перформанси, а RandomSempler ни треба за да го поделиме множеството на некоја рандом вредност.

In [ ]:
batch_size = 4
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

> **Setting Up Optimizer and Scheduler**

- Во овој дел дефинираме optimizer и scheduler. Како optimizer го користиме Adam и му ги задаваме параметрите на моделот и learning rate. Следно нешто што дефинираме е scheduler односно како optimizer-oт ќе го итерираме во torch моделот. Од transformers импортираме get_linear_schedule_with_warmup и тука го имаме optimizer кој го дефиниравме претходно.

In [ ]:
from transformers import AdamW
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 4
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
) 

> **Defining our Performance Metrics**

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

- Дефинираме функција за ф1 score function со preds и labels. Бидејќи preds е повеќедимензионално земаме argmax од дадена низа, го земаме максимумот и го правиме flatten односно го ставаме во 1 димензија. Истото го правиме и за labels.

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

- Дефинираме функција за пресметка на точноста по класи.

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

> **Creating our Training Loop**

- Бидејќи имавме рандом поделба на множествата, правиме импорт на рандом и секаде поставуваме seed_val = 17 со цел секаде да ни биди истиот рандом.

- За зачувавање на историјата од torch ни треба некој device. Cuda означува сервер ако постои го користиме него ако не користиме cpu локално.


In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


- Тука дефинираме функција за евалуација. Најпрво правиме eval на моделот. Потоа поставуваме loss функција и итерираме низ batch size. Tqdm е итератор низ различни структури. Тука итерираме низ dataloader_val. За секоја итерација во batch го поставуваме toDevice. Inputs ни се поделени во input_ids, attention_mask и labels. 
torch.no_grad() значи да ги нема градиенти бидејќи при backpropagation ова се прави прво. Имаме што loss ни враќа и кои логови ни ги враќа. Евалуацијата се врши на валидациско множество. Во prediction ги ставаме новите predictions, а тоа се логовите. Следно се true_vals , a тоа се label_ids. Надвор од for сметаме loss_val_avg, ги спојуваме двете predictions по редици исто така ги спојуваме и true_vals. На крајот ги враќаме loss_val_avg, predictions, true_vals.

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

- Следно е итерирање и тренирање на моделот. Од tqdm земаме notebook. Го тренираме моделот и пресметуваме loss_train_total. Дефинираме еден прогрес бар кој што е итератор низ dataloader_train. Ја пресметуваме новата loss функција и повикуваме backpropagation на loss. Ги поставуваме новите параметрите. Optimizer и scheduler одат еден чекор напред. Надвор од for пресметуваме loss_train_avg. Со помош на evaluate пресметуваме на dataloader_val val_loss, predictions и true_vals. И со f1_score_func пресметуваме f1 score на validation. Печатиме Validation loss и F1 Score.

In [ ]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    

    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    print(f'Validation loss: {val_loss}')
    print(f'F1 Score (weighted): {val_f1}')

Training loss: 2.3155523013194403



Validation loss: 2.1768344914077953
F1 Score (weighted): 0.27468642431876034


Training loss: 1.9813968373113209



Validation loss: 2.1633207357613142
F1 Score (weighted): 0.3009343465405143


Training loss: 1.7285172355651854



Validation loss: 2.2097732717064535
F1 Score (weighted): 0.3186216175975761


Training loss: 1.5275085290749868



Validation loss: 2.300377290719634
F1 Score (weighted): 0.3236762389312333



Evaluating our Model


- Како за крај се прави евалуација на моделот, односно се проверува колкава точност има истиот. Точноста на моделот се проверува преку делење на бројот на точни предикции со вкупниот број на предикции.

In [ ]:
accuracy_per_class(predictions, true_vals) 

Class: 12
Accuracy:669/1056

Class: 19
Accuracy:191/521

Class: 0
Accuracy:288/504

Class: 2
Accuracy:11/308

Class: 11
Accuracy:119/243

Class: 7
Accuracy:35/238

Class: 1
Accuracy:35/204

Class: 9
Accuracy:53/199

Class: 4
Accuracy:9/177

Class: 8
Accuracy:21/171

Class: 5
Accuracy:114/159

Class: 10
Accuracy:57/143

Class: 14
Accuracy:49/129

Class: 16
Accuracy:6/153

Class: 18
Accuracy:13/129

Class: 6
Accuracy:31/148

Class: 13
Accuracy:5/137

Class: 3
Accuracy:93/121

Class: 15
Accuracy:11/129

Class: 17
Accuracy:2/131

